In [1]:
import pandas as pd

location = []
pricing = []
features = []

DF_GA_RAW = pd.read_csv("RealEstate_Georgia.csv", chunksize=100)

#not big enought to need to handle chunks seperately
#so I can just concatenate them together
DF_GA = pd.concat(DF_GA_RAW)

for x, row in DF_GA.iterrows():
    
    #if there is a 0 number address, not valid
    if row['streetAddress'][0] == '0':
        continue
        
    #not looking for investment properties
    if row['homeType'] == 'LOT' or row['homeType'] == 'MULTI_FAMILY':
        continue
        
    ##will use living area instead of building (too many null values)
    #living area and living area value are always the same
    if row['price'] == 0:
        continue
        
    temp_id = int(row['id'].replace('-','') + str(x))
    location.append(
        {
            "id" : temp_id,
            "zipcode" : row['zipcode'],
            "address" : row['streetAddress'],
            "city" : row['city'],
            "county" : row['county']
        }
    )
    pricing.append(
        {
            "id" : temp_id,
            "price" : row['price'],
            "ppsqft" : row['pricePerSquareFoot'],
            "area" : row['livingArea']
        }
    )
    features.append(
        {
            "id" : temp_id,
            "bed" : row['bedrooms'],
            "bath" : row['bathrooms'],
            "levels" : row['levels'],
            "style" : row['homeType'].replace('_',' ').title(),
            "year" : row['yearBuilt'],
            "has_parking" : row['parking'],
            "has_garage" : row['hasGarage'],
            "garage_spaces" : row['garageSpaces'],
            "has_pool" : row['pool'],
        }
    )

In [2]:
import sqlite3
from sqlite3 import Error

try:
    conn = sqlite3.connect("fission")
except Error as e:
    print("Error occured")
print(conn)
cursor = conn.cursor()

create_location = """
CREATE TABLE IF NOT EXISTS location (
    id VARCHAR PRIMARY KEY NOT NULL,
    zipcode INTEGER NOT NULL,
    address TEXT NOT NULL,
    city TEXT NOT NULL,
    county TEXT NOT NULL
    
);
"""
create_pricing = """
CREATE TABLE IF NOT EXISTS pricing (
    id VARCHAR PRIMARY KEY NOT NULL,
    price INTEGER NOT NULL,
    ppsqft INTEGER NOT NULL,
    area INTEGER NOT NULL
);
"""
create_features = """
CREATE TABLE IF NOT EXISTS features (
    id VARCHAR PRIMARY KEY NOT NULL,
    bed FLOAT NOT NULL,
    bath FLOAT NOT NULL,
    levels TEXT NOT NULL,
    style TEXT NOT NULL,
    year INTEGER NOT NULL,
    has_parking INTEGER NOT NULL,
    has_garage INTEGER NOT NULL,
    garage_spaces FLOAT NOT NULL,
    has_pool INTEGER NOT NULL
);
"""
cursor.execute(create_location)
cursor.execute(create_pricing)
cursor.execute(create_features)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
print(cursor.fetchall())

[('features',), ('location',), ('pricing',)]


In [3]:
for x in location:
    temp_insert = (
        "INSERT INTO location VALUES"
        f"({x['id']},{x['zipcode']},\"{x['address']}\",\"{x['city']}\",\"{x['county']}\");"
    )
    cursor.execute(temp_insert)
cursor.execute("SELECT * FROM location LIMIT 10")
print(cursor.fetchall())

[('315031107854310', 31503, '103 Hog Creek Rd', 'Waycross', 'Brantley County'), ('31503766110821', 31503, '605 Ware St', 'Waycross', 'Ware County'), ('31503931261532', 31503, '1961 Mount Pleasant Rd', 'Waycross', 'Ware County'), ('315031107855983', 31503, '39 Joyce Rd', 'Waycross', 'Brantley County'), ('315032274213305', 31503, '3475 Lark Rdg', 'Waycross', 'Ware County'), ('315031107843076', 31503, '178 Wynton Cir', 'Waycross', 'Brantley County'), ('31503766111837', 31503, '1807 Gibbs St', 'Waycross', 'Ware County'), ('31503931264638', 31503, '1200 N Augusta Ave', 'Waycross', 'Ware County'), ('31503931245129', 31503, '1801 Creswell St', 'Waycross', 'Ware County'), ('315039313349510', 31503, '1303 Carswell Ave', 'Waycross', 'Ware County')]


In [4]:
cursor.execute("SELECT * FROM location LIMIT 10")
print(cursor.fetchall())

[('315031107854310', 31503, '103 Hog Creek Rd', 'Waycross', 'Brantley County'), ('31503766110821', 31503, '605 Ware St', 'Waycross', 'Ware County'), ('31503931261532', 31503, '1961 Mount Pleasant Rd', 'Waycross', 'Ware County'), ('315031107855983', 31503, '39 Joyce Rd', 'Waycross', 'Brantley County'), ('315032274213305', 31503, '3475 Lark Rdg', 'Waycross', 'Ware County'), ('315031107843076', 31503, '178 Wynton Cir', 'Waycross', 'Brantley County'), ('31503766111837', 31503, '1807 Gibbs St', 'Waycross', 'Ware County'), ('31503931264638', 31503, '1200 N Augusta Ave', 'Waycross', 'Ware County'), ('31503931245129', 31503, '1801 Creswell St', 'Waycross', 'Ware County'), ('315039313349510', 31503, '1303 Carswell Ave', 'Waycross', 'Ware County')]


In [5]:
for x in pricing:
    temp_insert = (
        "INSERT INTO pricing VALUES"
        f"({x['id']},{x['price']},{x['ppsqft']},{x['area']});"
    )
    cursor.execute(temp_insert)
cursor.execute("SELECT * FROM pricing LIMIT 10")
print(cursor.fetchall())

[('315031107854310', 169900, 74, 2302), ('31503766110821', 159000, 56, 2864), ('31503931261532', 45000, 62, 728), ('315031107855983', 165000, 102, 1620), ('315032274213305', 173500, 115, 1512), ('315031107843076', 349900, 152, 2309), ('31503766111837', 99500, 72, 1383), ('31503931264638', 129500, 69, 1874), ('31503931245129', 159995, 87, 1838), ('315039313349510', 59900, 46, 1300)]


In [7]:
for x in features:
    temp_insert = (
        "INSERT INTO features VALUES"
        f"({x['id']},{x['bed']},{x['bath']},{x['levels']},"
        f"\"{x['style']}\",{x['year']},{x['has_parking']},{x['has_garage']},"
        f"{x['garage_spaces']},{x['has_pool']});"
    )
    cursor.execute(temp_insert)
cursor.execute("SELECT * FROM features LIMIT 10")
print(cursor.fetchall())

[(3150376611082, 4.0, 2.0, 0, 'Single Family', 1910, 0, 0, 0.0, 0), (3150376611183, 3.0, 1.0, 0, 'Single Family', 1960, 0, 0, 0.0, 0), (3150393124512, 3.0, 3.0, 0, 'Single Family', 1922, 0, 0, 0.0, 1), (3150393126153, 1.0, 1.0, 0, 'Single Family', 1945, 1, 0, 0.0, 0), (3150393126463, 3.0, 2.0, 0, 'Single Family', 1940, 0, 0, 0.0, 0), (31503110784307, 3.0, 2.0, 0, 'Single Family', 1999, 1, 0, 0.0, 1), (31503110785431, 4.0, 3.0, 0, 'Single Family', 1985, 0, 0, 0.0, 0), (31503110785598, 2.0, 2.0, 0, 'Single Family', 1971, 0, 0, 0.0, 0), (31503227421330, 3.0, 2.0, 0, 'Single Family', 1980, 0, 0, 0.0, 0)]
